In [1]:
sandbox = False
get_access_only = False
if sandbox==False:
    userid = "NT1945"
    password='BT121299@'
    access_token = "3bf7af61-77c2-3f2e-b1c1-f4f5cb909db3"
    consumer_key = "ay4bObamN3KHOPzZjmpA7g9YDJwa"
    host = 'https://tradeapi.kotaksecurities.com/apim'
else:
    userid = "1T068"
    password='login@1'
    access_token = "ebe9c833-26c1-3402-bf80-c457e76a4da3"
    consumer_key = "a0s3dcOM4JgVNn3pB7fdLBFkJ4Ea"
    host = 'https://sbx.kotaksecurities.com/apim'

In [2]:
import get_otp
access_code=get_otp.get_otp()

In [3]:
#GENERATING OTP
from ks_api_client.ks_api import KSTradeApi
client = KSTradeApi(access_token = access_token, userid = userid, \
                consumer_key = consumer_key, ip = "127.0.0.1", app_id = "DefaultApplication", host = host)
client.login(password=password)

{'Success': {'Status': 'Disclaimer',
  'accessCodeTime': '1/29/2022 5:24:13 PM',
  'authLevel': 2,
  'biometric': 'N',
  'clientType': 'ONLINE',
  'disclaimer': {'disclaimer': 'POA', 'flag': 'M', 'linkId': 0},
  'emailId': 'maXXXXXXXXXXXXXXXse@gmail.com',
  'message': 'Authentication Successful.',
  'mpin': 'N',
  'nomineeFlag': 0,
  'oneTimeToken': 'D016DA921389C10E5C1ABF3167374C55F71CF74FE88C07',
  'phoneNo': 'XXXXXXX123',
  'redirect': {'host': 'https://tradeapi.kotaksecurities.com/apim',
   'port': 443},
  'userid': 'NT1945'}}

In [4]:
#OTP verification

if get_access_only==True:
    exit()

if sandbox==True:
    client.session_2fa(access_code = '1111')
else:
    client.session_2fa(access_code = access_code)

In [19]:
#Get token list

from datetime import datetime
import requests
#fno_url = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_FNO_'+datetime.now().strftime("%d_%m_%Y")+'.txt'
#cash_url = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_CASH_'+datetime.now().strftime("%d_%m_%Y")+'.txt'

fno_url = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_FNO_10_12_2021.txt'
Scash_url = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_CASH_10_12_2021.txt'

fno_tokens = requests.get(fno_url).text
cash_tokens = requests.get(cash_url).text

fno_tokens = fno_tokens.split('\n')
fno_tokens = [i.strip().split('|') for i in fno_tokens]

cash_tokens = cash_tokens.split('\n')
cash_tokens = [i.strip().split('|') for i in cash_tokens]

if len(fno_tokens) <= 1 or len(cash_tokens) <= 1:
    assert True==False, "Token list not valid"

import pandas as pd

fno_df = pd.DataFrame( fno_tokens[1:], columns=fno_tokens[0])
cash_df = pd.DataFrame( cash_tokens[1:], columns=cash_tokens[0])

103009
7505


In [20]:
#Get cash tokens
token = 'NIFTY 50'
banknifty_token = int(cash_df[cash_df.instrumentName==token].instrumentToken.values[0])
print(f"{token} TOKEN:",banknifty_token)

NIFTY 50 TOKEN: 11721


In [21]:
from helper import *
atm = get_atm(banknifty_token, client)
print(atm)
fno_df[ (fno_df.instrumentName == 'NIFTY') & (fno_df.strike==str(int(atm)))][0:2]

17100


,instrumentToken,instrumentName,name,lastPrice,expiry,strike,tickSize,lotSize,instrumentType,segment,exchange,isin,multiplier,exchangeToken,optionType
14952,19829,NIFTY,,254.1,03FEB22,17100,0.050000,50,OI,FO,NSE,,1,45536,CE
14953,19833,NIFTY,,241.1,03FEB22,17100,0.050000,50,OI,FO,NSE,,1,45537,PE


In [ ]:
# get ltp and atm

def get_atm():
    banknity_ltp = client.quote(instrument_token = banknifty_token, quote_type = "LTP")
    if 'success' in banknity_ltp:
        banknity_ltp = banknity_ltp['success'][0]['lastPrice']
    else:
        assert True==False

    banknity_ltp = float(banknity_ltp)
    banknity_atm = round(banknity_ltp, -2)

    print('BANK NIFTY ATM:', banknity_atm)
    return int(banknity_atm)

banknity_atm = get_atm()

In [ ]:
client.quote(instrument_token = 21272, quote_type='LTP')

In [ ]:
fno_df[fno_df.instrumentName == 'BANKNIFTY']

In [ ]:
# get pe ce tokens for a strike
def get_pe_ce_token(banknity_atm, fno_df):
    fno_df = fno_df[ (fno_df.instrumentName == 'NIFTY50') & (fno_df.strike==str(int(banknity_atm)))][0:2]
    banknity_token_pe = int(fno_df[fno_df.optionType=='PE']['instrumentToken'].values[0])
    banknity_token_ce = int(fno_df[fno_df.optionType=='CE']['instrumentToken'].values[0])

    print(f"BANK NIFTY {banknity_atm} PE TOKEN:",banknity_token_pe)
    print(f"BANK NIFTY {banknity_atm} CE TOKEN:",banknity_token_ce)
    return banknity_token_pe,banknity_token_ce

banknity_token_pe, banknity_token_ce = get_pe_ce_token(banknity_atm, fno_df)
banknity_token_pe, banknity_token_ce

In [ ]:
def get_quote(instrument_token):
    ltp = client.quote(instrument_token = instrument_token, quote_type = "LTP")
    if 'success' in ltp:
        ltp = ltp['success'][0]['lastPrice']
    else:
        assert True==False

    return ltp

print(f'banknity_ce {banknity_atm} LTP', get_quote(banknity_token_ce))
print(f'banknity_pe {banknity_atm} LTP', get_quote(banknity_token_pe))

In [ ]:
from datetime import datetime

old_second = 70

while True:
    
    hour, minute, second = [int(i) for i in datetime.now().strftime("%H.%M.%S").split('.')]
    
    if(old_second!=second):
        print(hour,minute,second)

        #update time
        old_second=second
        
        #Before 9:30am
        if(hour==9 and minute==29 and int(second)==58):
            #get banknifty atm
            banknity_atm = get_atm()
            
            #get pe and ce token at that atm
            banknity_token_pe, banknity_token_ce = get_pe_ce_token(banknity_atm, fno_df)

            #initialize hit flags
            banknifty_ce_hit = False
            banknifty_pe_hit = False
            banknifty_ce_wrong = False
            banknifty_pe_wrong = False

        #At 9:30am
        if(hour==9 and minute==30 and int(second)==0):
            #get ce and pe quote
            banknifty_ce_ltp_930 = get_quote(banknity_token_ce)
            banknifty_pe_ltp_930 = get_quote(banknity_token_pe)

            banknifty_ce_ex = 0.9*banknifty_ce_ltp_930
            banknifty_pe_ex = 0.9*banknifty_pe_ltp_930
        
        #After 9:30am
        if(hour>=9 and minute>=30):
            banknifty_ce_ltp = get_quote(banknity_token_ce)
            banknifty_pe_ltp = get_quote(banknity_token_pe)

            if (banknifty_ce_ltp<=banknifty_ce_ex and banknifty_ce_hit==False):
                #sell ce
                banknifty_ce_hit = True
                pass

            if (banknifty_ce_hit==True and banknifty_ce_ltp>=1.25*banknifty_ce_ex):
                #buy ce
                banknifty_ce_wrong = True
                pass

            if (banknifty_pe_ltp<=banknifty_pe_ex and banknifty_pe_hit==False):
                #sell pe
                banknifty_pe_hit = True
                pass

            if (banknifty_pe_hit==True and banknifty_pe_ltp>=1.25*banknifty_pe_ex):
                #buy pe
                banknifty_pe_wrong = True
                pass

            #At 3pm
            if(hour==15 and minute==0 and int(second)==0):
                if( banknifty_ce_hit==True and banknifty_ce_wrong==False ):
                    #buy ce
                    pass

                if( banknifty_pe_hit==True and banknifty_pe_wrong==False ):
                    #buy pe
                    pass            

        

        

In [ ]:
from datetime import datetime
datetime.now().strftime("%H:%M:%S")

In [ ]:
from helper import log_info
with open('harsh.txt', 'a') as f:
    log_info(f,'SELL BLAH BLAH', 'SELL')

In [3]:
!python3 temp.py --error

error


In [14]:
import os
p = os.system('python3 temp.py --error')
print(p)
print(type(p))

error
256
<class 'int'>


Ohh noo
